In [1]:
pip install yfinance

     |████████████████████████████████| 63 kB 877 kB/s 
     |████████████████████████████████| 6.4 MB 49.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
pip install ipeadatapy

In [3]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime
import re
import yfinance as yf
import ipeadatapy as ipea

# Introdução

O objetivo deste módulo é dispor sobre a metodologia de cálculo do Custo de
Capital nos processos de Revisão Tarifária Periódica – RTP da concessionária dos serviços públicos de abastecimento de água e esgotamento sanitário do Distrito Federal.

# Metodologia

## Estrutura de Capital

A estrutura de capital regulatória é definida pela participação de capital próprio e de capital de terceiros na concessionária. É utilizada para o cálculo do Custo de Capital.

### Cálculo do Capiral Prórpio

O Capital Próprio (CP) será a média do Patrimônio Líquido da Caesb, dos últimos 4 exercícios, obtida das demonstrações contábeis publicadas pela empresa, em data prévia mais próxima ao último dia útil do ano imediatamente anterior à RTP em processamento. 

### Cálculo do Capital de Terceiros
O Capital de Terceiros (CT) da Caesb será a média, dos últimos 4 exercícios, da soma de todos os passivos onerosos, constituídos por saldos de empréstimos, de financiamentos e outros instrumentos financeiros similares, de curto ou longo prazos, deduzidos do Caixa e Equivalentes de Caixa. Estes dados devem ser obtidos das demonstrações financeiras publicadas pela concessionária, em data prévia mais próxima ao último dia útil do ano imediatamente anterior à RTP em processamento.

### Fonte
As informações para o cálculo da Estrutura de Capital podem ser acessadas pelo site:

https://www.caesb.df.gov.br/empresa/governanca-corporativa/balancos-e-relatorios/demonstracoes-financeiras.html

## Cálculo da Estrutura de Capital regulatória
A estrutura de capital regulatória é composta das seguintes partes:
* Participação de capital próprio $W_e$
  $$ W_e = \frac{CP}{CP + CT} $$
* Participação de capital de terceiros $W_d$
$$ W_d = \frac{CT}{CP+CT}$$

onde

$CP$ representa o capital próprio da empresa, que para fins de cálculo será utilizado a média dos últimos 4 exercícios do Patrimônio líquido da CAESB.

e

$CT$ reprenseta o capital de terceiro, que para fins de cálcula será utilizado a média dos últimos 4 exercícios da soma de todos os passivos onerosos.

**Estutura de Capital da Caesb**
https://docs.google.com/spreadsheets/d/1RaLNFrfwvvBrScjbEhH2jo_DC_1J7YJR05eP5U_OvXI/edit#gid=0&range=B9:F14

In [4]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('Estrutura de Capital da Caesb').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

estrutura_caesb = pd.DataFrame.from_records(rows, index = [0])
estrutura_caesb.columns = estrutura_caesb.iloc[0]
estrutura_caesb = estrutura_caesb.iloc[1:,1:]


[['Descricao', '2019', '2018', '2017', '2016'], ['Emprestimos e financiamentos - curto prazo', '203072', '235816', '219483', '139127'], ['Emprestimos e financiamentos - longo prazo', '704029', '582600', '411862', '459976'], ['Caixa e equivalentes', '311278', '190866', '143004', '84482'], ['Divida Liquida', '595823', '627550', '488341', '514621'], ['Patrimonio Liquido', '1404805', '1244028', '1421598', '1283175']]


In [5]:
divida_liq = np.array(estrutura_caesb[3:4][["2019","2018","2017","2016"]],
                      dtype=int)
patrimonio_liq = np.array(estrutura_caesb[4:5][["2019","2018","2017","2016"]],
                          dtype=int)

cap_proprio = patrimonio_liq/(patrimonio_liq + divida_liq)
media_cap_prop = cap_proprio.mean()

cap_terceiros = divida_liq/(patrimonio_liq + divida_liq)
media_cap_terc = cap_terceiros.mean()

## Custo de Capital

Para o cálculo do custo de capital, será utilizado o método *Weighted Average Cost of Capital* (WACC), que considera a média ponderada dos custos do capital próprio e do capital de terceiros, conforme apresentado na fórmula a seguir:

$$ WACC = K_{er} \times W_{e} + K_d(1 - imp)W_{d} $$

onde

$WACC$: Custo Médio Ponderado de Capital

$K_{er}$: custo do capital próprio;

$W_e$: participação do capital próprio;

$K_d$: custo do capital de terceiros;

$imp$: soma das alíquotas dos impostos incidentes sobre o lucro líquido das
empresas (9%);

$W_d$: participação do capital de terceiros.

### Custo do Capital Próprio

Modelo de precificação de ativos financeiros (CAPM)

$$ K_{er} = rf + \beta(E(r_m)_r - rf) + \alpha_{BR}$$

Sendo:

$K_{er} := $ custo de capital próprio;

$E(r_m)_r := $ retorno do mercado descontado a inflação;

$\beta := $ coeficiente de risco sistemático;

$rf := $ taxa livre de risco;

$\alpha_{BR} :=$ Risco-País do Brasil


#### Taxa Livre de Risco
A taxa livre de risco (rf) será resultado da média aritmética simples dos 
rendimentos correntes mensais de mercado (*current market yields*) dos títulos do governo 
americano (*10-Year Treasury Inflation-Indexed Security, Constant Maturity - DFII10*), 
de maturidade de 10 anos, indexados à inflação, divulgado pelo Federal Reserve (Banco 
Central dos Estados Unidos da América). 
Para o cálculo da média, devem ser utilizados os rendimentos dos últimos 180 
meses, a contar do último dia útil do ano anterior à RTP em processamento. 

Fonte: Federal Reserve


In [6]:
data_inicio = '2005-01-01'
data_final = '2019-12-31'

In [7]:
DFII10 = web.DataReader('DFII10', 'fred', start = data_inicio, end = data_final)

In [8]:
DFII10 = DFII10.resample('M').mean()

In [9]:
TX_livre_risco = np.array(DFII10).mean()/100

#### Retorno do Mercado
Para obter o retorno real do mercado, desconta-se o efeito da inflação por meio da seguinte fórmula:

$$E(r_m)_r = \frac{1+E(r_m)_n}{1+i} - 1 $$

Sendo:

$E(r_m)_r$: retorno do mercado descontada a inflação;

$E(r_m)_n$: retorno nominal do mercado;

$i$: inflação anual média do período

##### Retorno Nominal do Mercado

A taxa de retorno de mercado nominal deve ser obtida por meio da seguinte
fórmula:

$$E(r_m)_n = \left(\frac{Pm_t}{Pm_{t-10}}\right)^\frac{1}{10} - 1 $$

Sendo:

$Pm_t$: Preço (cotação) do índice de mercado *S&P 500* no último dia útil do ano $t$

$t$:ano imediatamente anterior à RTP em processamento

In [10]:
data_inicio = "2009-12-31"
data_final = "2020-01-01"

SP500 = yf.Ticker("^GSPC").history(start=data_inicio, end=data_final)

In [11]:
retorno_nominal = (SP500['Close'][-1] / SP500['Close'][0]) ** (1/10) - 1

##### Variação da Inflação

A taxa de inflação anual média (i) do período será a média anual do índice CPI (*Consumers Price Index*) para o período de 120 meses anteriores ao último dia útil do ano anterior à RTP em processamento, a qual será obtida por meio da seguinte fórmula:

$$ i=\left(\frac{CPI_t}{CPI_{t-10}}\right)^\frac{1}{10} - 1$$

Sendo:

$CPI_t$: valor do índice que mede a inflação dos EUA (CPI), no último dia útil do ano imediatamente anterior à RTP em processamento;

$t$: ano imediatamente anterior à RTP em processamento.

In [12]:
data_inicio = '2009-12-01'
data_final = '2019-12-31'
CPI_US = web.DataReader('CPIAUCSL', 'fred', start = data_inicio, end = data_final)

In [13]:
inflacao10 = (CPI_US['CPIAUCSL'][-1] / CPI_US['CPIAUCSL'][0]) ** (1/10) - 1

##### Retorno Real do Mercado

In [14]:
retorno_real = (retorno_nominal + 1) / (inflacao10 + 1) - 1

#### Coeficiente Beta ($β$)

O coeficiente beta ($β$) é uma medida de risco sistemático, o qual considera a variação do valor de um ativo em relação à variação do mercado, obtido por meio da seguinte fórmula:

$$ β = \frac{Cov(Rm_d;Ra_d)}{Var(Rm_d)}$$

sendo,

$Rm_d$: Retorno diário logarítmico do mercado (*S&P500*);

$𝑅𝑎_d$: Retorno diário logarítmico da ação da empresa; 

$𝑃𝑎_d$: Preço da ação no dia d; 

$Pa_{d-1}$: Preço da ação no dia d-1;

$d$: dia atual, variando entre os últimos 120 meses, a contar do último dia do 
ano anterior à RTP em processamento; 

$d-1$: dia imediatamente anterior ao dia atual.

**Seleção das empresas para o cálculo do Coeficiente Beta (β)** 

Para o cálculo do Coeficiente Beta (β) serão consideradas as empresas listadas nas bolsas norte-americanas NYSE (New York Stock Exchange) e Nasdaq, que prestam serviços de abastecimento de água e de esgotamento sanitário nos Estados Unidos da América. Segue abaixo a lista de empresas selecionadas:

1. American States Water Company (NYSE:AWR)
2. American Water Works Company, Inc. (NYSE:AWK)
3. Artesian Resources Corporation, Inc. (NasdaqGS:ARTN.A)
4. California Water Service Group (NYSE:CWT)
5. Middlesex Water Company (NasdaqGS:MSEX)
6. New England Service Company, Inc. (OTCPK:NESW)
7. The York Water Company (NasdaqGS:YORW)

In [15]:
sp500_close = SP500['Close']
log_retorno_sp500 = np.log(sp500_close / sp500_close.shift())

In [16]:
inicio = "2009-12-31"
fim = "2020-01-01"

##### American States Water Company - NYSE:AWR

In [17]:
AWR = yf.Ticker("AWR").history(start=inicio, end=fim)
AWR = AWR['Close']
log_retorno = np.log(AWR/AWR.shift())

In [18]:
X = np.array(log_retorno_sp500[1:]).reshape(-1, 1)
Y = np.array(log_retorno[1:]).reshape(-1, 1)

model = LinearRegression().fit(X, Y)
beta_AWR = model.coef_

##### American Water Works Company, Inc - NYSE:AWK

In [19]:
AWK = yf.Ticker("AWK").history(start=inicio, end=fim)
AWK = AWK['Close']
log_retorno = np.log(AWK/AWK.shift())

In [20]:
Y = np.array(log_retorno[1:]).reshape(-1,1)

model = LinearRegression().fit(X, Y)
beta_AWK = model.coef_

##### Artesian Resources Corporation, Inc - NASDAQ:ARTN.A

In [21]:
ARTNA = yf.Ticker("ARTNA").history(start=inicio, end=fim)
ARTNA = ARTNA['Close']
log_retorno = np.log(ARTNA/ARTNA.shift())

In [22]:
Y = np.array(log_retorno[1:]).reshape(-1,1)

model = LinearRegression().fit(X,Y)
beta_ARTNA = model.coef_

##### California Water Service Group - NYSE:CWT

In [23]:
CWT = yf.Ticker("CWT").history(start=inicio, end=fim)
CWT = CWT['Close']
log_retorno = np.log(CWT/CWT.shift())

In [24]:
Y = np.array(log_retorno[1:]).reshape(-1,1)

model = LinearRegression().fit(X,Y)
beta_CWT = model.coef_

##### Middlesex Water Company - NasdaqGS:MSEX

In [25]:
MSEX = yf.Ticker("MSEX").history(start=inicio, end=fim)
MSEX = MSEX['Close']
log_retorno = np.log(MSEX/MSEX.shift())

In [26]:
Y = np.array(log_retorno[1:]).reshape(-1,1)

model = LinearRegression().fit(X,Y)
beta_MSEX = model.coef_

##### New England Service Company, Inc. - OTCPK:NESW

In [27]:
SJW = yf.Ticker("SJW").history(start=inicio, end=fim)
SJW = SJW['Close']
log_retorno = np.log(SJW/SJW.shift())

In [28]:
Y = np.array(log_retorno[1:]).reshape(-1,1)

model = LinearRegression().fit(X,Y)
beta_SJW = model.coef_

##### The York Water Company (NasdaqGS:YORW)

In [29]:
YORW = yf.Ticker("YORW").history(start=inicio, end=fim)
YORW = YORW['Close']
log_retorno = np.log(YORW/YORW.shift())

In [30]:
Y = np.array(log_retorno[1:]).reshape(-1,1)

model = LinearRegression().fit(X,Y)
beta_YORW = model.coef_

##### Cálculo do Beta

Calcular o β médio, por meio da média aritmética simples dos betas de cada empresa.

In [31]:
beta = np.array([beta_AWR, beta_AWK, beta_ARTNA, beta_CWT, beta_MSEX, beta_SJW, beta_YORW])
beta_medio = beta.mean()

#### Prêmio de Risco País

O prêmio de risco país adotado será obtido pela média aritmética do valor diário do índice EMBI+Brasil.

Diponibilizado pelo IPEA e calculado pelo JP Morgan.

Deverá ser adotada a série histórica dos últimos 180 meses, a contar do último dia útil do ano anterior à RTP em processamento. 


In [32]:
ipea_data = ipea.list_series()
ipea_data[ipea_data.NAME == 'EMBI + Risco-Brasil']

,CODE,NAME
8000,JPM366_EMBI366,EMBI + Risco-Brasil


In [33]:
data_inicio = '2004-12-31'
data_final = '2019-12-31'

risco_brasil = ipea.timeseries("JPM366_EMBI366").dropna()
risco_brasil = risco_brasil.loc[data_inicio:data_final]
risco_brasil = np.array(risco_brasil['VALUE (-)']).mean()/10000

In [34]:
ipea.timeseries('JPM366_EMBI366',   yearGreaterThan=2016, yearSmallerThan=2019)

,YEAR,DAY,MONTH,CODE,RAW DATE,VALUE (-)
DATE,,,,,,
2017-01-02,2017,2,1,JPM366_EMBI366,2017-01-02T00:00:00-02:00,328.0
2017-01-03,2017,3,1,JPM366_EMBI366,2017-01-03T00:00:00-02:00,325.0
2017-01-04,2017,4,1,JPM366_EMBI366,2017-01-04T00:00:00-02:00,311.0
2017-01-05,2017,5,1,JPM366_EMBI366,2017-01-05T00:00:00-02:00,302.0
2017-01-06,2017,6,1,JPM366_EMBI366,2017-01-06T00:00:00-02:00,301.0
...,...,...,...,...,...,...
2018-12-24,2018,24,12,JPM366_EMBI366,2018-12-24T00:00:00-02:00,276.0
2018-12-26,2018,26,12,JPM366_EMBI366,2018-12-26T00:00:00-02:00,273.0
2018-12-27,2018,27,12,JPM366_EMBI366,2018-12-27T00:00:00-02:00,274.0


#### Cálculo do Custo de Capital Próprio

In [35]:
custo_cap_prop = TX_livre_risco + beta_medio*(retorno_real - TX_livre_risco) + risco_brasil

### Custo Capital de Terceiros

O Custo de Capital de Terceiros (Kd) é a média histórica da parcela de juros
real da Taxa de Juros de Longo Prazo (TJLP), dos últimos 240 meses, a contar do último dia útil do ano anterior à RTP em processamento.

In [36]:
from numpy.core.arrayprint import dtype_is_implied
worksheet = gc.open('TJLP').sheet1

rows = worksheet.get_all_values()
print(rows)

TJLP = pd.DataFrame.from_records(rows, index=[0], coerce_float=True)
TJLP.columns = TJLP.iloc[0]
TJLP = TJLP.iloc[1:-1,1:]
custo_cap_terc = np.array(TJLP['256 - Taxa de juros de longo prazo - TJLP - % a.a.'],
                          dtype=float).mean()/100

[['Data', '256 - Taxa de juros de longo prazo - TJLP - % a.a.'], ['01/2000', '12'], ['02/2000', '12'], ['03/2000', '12'], ['04/2000', '11'], ['05/2000', '11'], ['06/2000', '11'], ['07/2000', '10.25'], ['08/2000', '10.25'], ['09/2000', '10.25'], ['10/2000', '9.75'], ['11/2000', '9.75'], ['12/2000', '9.75'], ['01/2001', '9.25'], ['02/2001', '9.25'], ['03/2001', '9.25'], ['04/2001', '9.25'], ['05/2001', '9.25'], ['06/2001', '9.25'], ['07/2001', '9.5'], ['08/2001', '9.5'], ['09/2001', '9.5'], ['10/2001', '10'], ['11/2001', '10'], ['12/2001', '10'], ['01/2002', '10'], ['02/2002', '10'], ['03/2002', '10'], ['04/2002', '9.5'], ['05/2002', '9.5'], ['06/2002', '9.5'], ['07/2002', '10'], ['08/2002', '10'], ['09/2002', '10'], ['10/2002', '10'], ['11/2002', '10'], ['12/2002', '10'], ['01/2003', '11'], ['02/2003', '11'], ['03/2003', '11'], ['04/2003', '12'], ['05/2003', '12'], ['06/2003', '12'], ['07/2003', '12'], ['08/2003', '12'], ['09/2003', '12'], ['10/2003', '11'], ['11/2003', '11'], ['12/2003

### Cálculo do Custo de Capital (WACC)

In [37]:
wacc = custo_cap_prop * media_cap_prop + custo_cap_terc * (1 - 0.09) * media_cap_terc

In [38]:
print("Taxa livre de Risco = {:.2%}".format(TX_livre_risco))
print("Retorno do mercado nominal = {:.2%}".format(retorno_nominal))
print("Inflação americana = {:.2%}".format(inflacao10))
print("Retorno do mercado Real = {:.2%}".format(retorno_real))
print("Beta = {:.2}".format(beta_medio))
print("Risco País = {:.2%}".format(risco_brasil))

print("Custo Capital Próprio (Real) = {:.2%}".format(custo_cap_prop))
print("Custo do Capital de Terceiros (Real) = {:.2%}".format(custo_cap_terc))

print("Parcela do capital próprio = {:.2%}".format(media_cap_prop))
print("Parcela do capital de terceiros = {:.2%}".format(media_cap_terc))

Taxa livre de Risco = 0.93%
Retorno do mercado nominal = 11.22%
Inflação americana = 1.74%
Retorno do mercado Real = 9.32%
Beta = 0.69
Risco País = 2.63%
Custo Capital Próprio (Real) = 9.37%
Custo do Capital de Terceiros (Real) = 7.47%
Parcela do capital próprio = 70.62%
Parcela do capital de terceiros = 29.38%


In [39]:
print("WACC = {:.2%}".format(wacc))

WACC = 8.61%
